Dress recomendation system based on the venue

Installing the dependencies

In [2]:
pip install pandas

     ---------------------------------------- 11.6/11.6 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 347.8/347.8 KB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 4.7 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 KB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install seaborn

     -------------------------------------- 294.9/294.9 KB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 7.8/7.8 MB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 KB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 6.9 MB/s eta 0:00:00
     -------------------------------------- 111.1/111.1 KB 6.3 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 6.1 MB/s eta 0:00:00
     -------------------------------------- 211.8/211.8 KB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install scikit-learn

     ---------------------------------------- 11.2/11.2 MB 5.2 MB/s eta 0:00:00
     ---------------------------------------- 46.2/46.2 MB 3.9 MB/s eta 0:00:00
     -------------------------------------- 307.7/307.7 KB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install --upgrade pip

     ---------------------------------------- 1.8/1.8 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Load the datasets
def load_data():
    clothing_items = pd.read_csv("clothing_items_csv.txt")
    venues = pd.read_csv("venues_csv.txt")
    compatibility_matrix = pd.read_csv("compatibility_matrix_csv.txt")
    outfit_combinations = pd.read_csv("outfit_combinations_csv.txt")
    recommendation_rules = pd.read_csv("recommendation_rules_csv.txt")
    image_references = pd.read_csv("image_references_csv.txt")
    
    # Parse item lists in outfit combinations
    outfit_combinations['items_list'] = outfit_combinations['items'].apply(lambda x: x.split(','))
    outfit_combinations['venue_ids_list'] = outfit_combinations['venue_ids'].apply(lambda x: x.split(','))
    
    return clothing_items, venues, compatibility_matrix, outfit_combinations, recommendation_rules, image_references

In [22]:
def explore_data(clothing_items, venues, compatibility_matrix):
    """Explore the data to understand relationships"""
    # Distribution of formality levels in clothing items
    plt.figure(figsize=(10, 6))
    sns.countplot(x='formality_level', data=clothing_items)
    plt.title('Distribution of Clothing Formality Levels')
    plt.savefig('clothing_formality_dist.png')
    plt.close()
    
    # Distribution of formality levels in venues
    plt.figure(figsize=(10, 6))
    sns.countplot(x='formality_level', data=venues)
    plt.title('Distribution of Venue Formality Levels')
    plt.savefig('venue_formality_dist.png')
    plt.close()
    
    # Heatmap of compatibility scores
    pivot_comp = compatibility_matrix.pivot_table(
        index='venue_id', 
        columns='item_id', 
        values='appropriateness_score', 
        aggfunc='mean'
    )
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_comp, cmap='YlGnBu', annot=True, fmt='.1f')
    plt.title('Venue-Item Compatibility Scores')
    plt.savefig('compatibility_heatmap.png')
    plt.close()
    
    return pivot_comp

In [23]:
def preprocess_data(clothing_items, venues):
    """Transform categorical features into numeric representations"""
    # For clothing items
    cat_features_items = ['category', 'color', 'pattern', 'material', 'season', 'gender_category']
    enc_items = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_features_items = enc_items.fit_transform(clothing_items[cat_features_items])
    encoded_df_items = pd.DataFrame(encoded_features_items, 
                                   columns=enc_items.get_feature_names_out(cat_features_items))
    
    # Add formality level as a numeric feature
    encoded_df_items['formality_level'] = clothing_items['formality_level']
    
    # For venues
    cat_features_venues = ['typical_weather', 'time_of_day']
    enc_venues = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_features_venues = enc_venues.fit_transform(venues[cat_features_venues])
    encoded_df_venues = pd.DataFrame(encoded_features_venues, 
                                    columns=enc_venues.get_feature_names_out(cat_features_venues))
    
    # Add formality level as a numeric feature
    encoded_df_venues['formality_level'] = venues['formality_level']
    
    return encoded_df_items, encoded_df_venues, enc_items, enc_venues

In [24]:
def build_compatibility_model(compatibility_matrix, clothing_items, venues, encoded_df_items, encoded_df_venues):
    """Build a random forest model to predict compatibility scores"""
    # Prepare training data
    training_data = []
    
    for _, row in compatibility_matrix.iterrows():
        venue_id = row['venue_id']
        item_id = row['item_id']
        score = row['appropriateness_score']
        
        # Get venue features
        venue_idx = venues[venues['venue_id'] == venue_id].index[0]
        venue_features = encoded_df_venues.iloc[venue_idx].values
        
        # Get item features
        item_idx = clothing_items[clothing_items['item_id'] == item_id].index[0]
        item_features = encoded_df_items.iloc[item_idx].values
        
        # Combine features
        combined_features = np.concatenate([venue_features, item_features])
        training_data.append((combined_features, score))
    
    # Split into X and y
    X = np.array([item[0] for item in training_data])
    y = np.array([item[1] for item in training_data])
    
    # Train a random forest regressor
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    return model

In [25]:
def build_outfit_recommender(clothing_items, encoded_df_items, outfit_combinations):
    """Build a nearest neighbors model for outfit recommendations"""
    # Create a mapping from item_id to its features
    item_features_map = {}
    for idx, row in clothing_items.iterrows():
        item_id = row['item_id']
        item_features = encoded_df_items.iloc[idx].values
        item_features_map[item_id] = item_features
    
    # Create outfit feature vectors by averaging item features
    outfit_features = []
    outfit_ids = []
    
    for _, row in outfit_combinations.iterrows():
        outfit_id = row['outfit_id']
        items_list = row['items_list']
        
        # Compute average features across all items in the outfit
        item_features_list = [item_features_map[item_id] for item_id in items_list if item_id in item_features_map]
        if item_features_list:
            avg_features = np.mean(item_features_list, axis=0)
            outfit_features.append(avg_features)
            outfit_ids.append(outfit_id)
    
    # Build a nearest neighbors model
    nn_model = NearestNeighbors(n_neighbors=3, algorithm='ball_tree')
    nn_model.fit(outfit_features)
    
    return nn_model, outfit_features, outfit_ids

In [26]:
def feature_importance(model, encoded_df_items, encoded_df_venues):
    """Analyze feature importance for the compatibility model"""
    feature_names = list(encoded_df_venues.columns) + list(encoded_df_items.columns)
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(12, 8))
    plt.title('Feature Importances for Outfit Compatibility')
    plt.bar(range(len(indices[:15])), importances[indices[:15]], align='center')
    plt.xticks(range(len(indices[:15])), [feature_names[i] for i in indices[:15]], rotation=90)
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    return [(feature_names[i], importances[i]) for i in indices[:10]]

In [27]:
def recommend_outfit_for_venue(venue_id, venues, clothing_items, model, outfit_combinations, 
                              nn_model, outfit_features, outfit_ids, encoded_df_venues, encoded_df_items,
                              recommendation_rules):
    """Recommend outfits for a given venue"""
    # Get venue information
    venue_info = venues[venues['venue_id'] == venue_id].iloc[0]
    venue_idx = venues[venues['venue_id'] == venue_id].index[0]
    venue_features = encoded_df_venues.iloc[venue_idx].values
    
    print(f"Recommendations for venue: {venue_info['venue_name']} (Formality Level: {venue_info['formality_level']})")
    
    # 1. Direct outfit recommendations based on outfits marked for this venue
    direct_outfits = outfit_combinations[outfit_combinations['venue_ids'].str.contains(venue_id)]
    print("\nPre-defined outfits for this venue:")
    if not direct_outfits.empty:
        for _, outfit in direct_outfits.iterrows():
            items_list = outfit['items_list']
            item_names = [clothing_items[clothing_items['item_id'] == item_id]['item_name'].values[0] 
                         for item_id in items_list if item_id in clothing_items['item_id'].values]
            print(f"- {outfit['name']} (Score: {outfit['overall_score']}): {', '.join(item_names)}")
    else:
        print("No pre-defined outfits found for this venue.")
    
    # 2. Find similar outfits based on venue formality
    # Create a query vector representing the venue's formality level
    query_vector = np.zeros_like(outfit_features[0])
    formality_idx = list(encoded_df_venues.columns).index('formality_level')
    query_vector[formality_idx] = venue_info['formality_level']
    
    # Find nearest outfit neighbors
    distances, indices = nn_model.kneighbors([query_vector])
    
    print("\nRecommended outfits based on similarity to venue formality:")
    for idx in indices[0]:
        outfit_id = outfit_ids[idx]
        outfit = outfit_combinations[outfit_combinations['outfit_id'] == outfit_id].iloc[0]
        items_list = outfit['items_list']
        item_names = [clothing_items[clothing_items['item_id'] == item_id]['item_name'].values[0] 
                     for item_id in items_list if item_id in clothing_items['item_id'].values]
        print(f"- {outfit['name']} (Score: {outfit['overall_score']}): {', '.join(item_names)}")
    
    # 3. Item-by-item compatibility check for top-rated individual items
    compatible_items = []
    
    for idx, item in clothing_items.iterrows():
        item_id = item['item_id']
        item_features = encoded_df_items.iloc[idx].values
        
        # Combine venue and item features
        combined_features = np.concatenate([venue_features, item_features])
        
        # Predict compatibility
        score = model.predict([combined_features])[0]
        compatible_items.append((item_id, score, item['item_name'], item['category']))
    
    # Sort by score and category
    compatible_items.sort(key=lambda x: (-x[1], x[3]))
    
    # Group by category and show top item for each
    categories = {}
    for item in compatible_items:
        if item[3] not in categories:
            categories[item[3]] = item
    
    print("\nRecommended individual items by category:")
    for category, item in categories.items():
        print(f"- {category}: {item[2]} (Compatibility Score: {item[1]})")
    
    # 4. Check recommendation rules
    print("\nSpecial recommendations based on rules:")
    venue_rules = recommendation_rules[recommendation_rules['venue_id'] == venue_id]
    if not venue_rules.empty:
        for _, rule in venue_rules.iterrows():
            inappropriate_id = rule['inappropriate_item_id']
            inappropriate_name = clothing_items[clothing_items['item_id'] == inappropriate_id]['item_name'].values[0] if inappropriate_id else ""
            
            recommended_id = rule['recommended_item_id']
            if pd.notna(recommended_id):
                recommended_name = clothing_items[clothing_items['item_id'] == recommended_id]['item_name'].values[0]
                print(f"- {rule['recommendation_text']} (Replace {inappropriate_name} with {recommended_name})")
            else:
                print(f"- {rule['recommendation_text']}")
    else:
        print("No specific rules found for this venue.")

def evaluate_model_accuracy(compatibility_matrix, clothing_items, venues, encoded_df_items, encoded_df_venues, model):
    """Evaluate the accuracy of the compatibility prediction model using cross-validation"""
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
    
    # Prepare data for evaluation
    X = []
    y = []
    
    for _, row in compatibility_matrix.iterrows():
        venue_id = row['venue_id']
        item_id = row['item_id']
        score = row['appropriateness_score']
        
        # Get venue features
        venue_idx = venues[venues['venue_id'] == venue_id].index[0]
        venue_features = encoded_df_venues.iloc[venue_idx].values
        
        # Get item features
        item_idx = clothing_items[clothing_items['item_id'] == item_id].index[0]
        item_features = encoded_df_items.iloc[item_idx].values
        
        # Combine features
        combined_features = np.concatenate([venue_features, item_features])
        X.append(combined_features)
        y.append(score)
    
    X = np.array(X)
    y = np.array(y)
    
    # Cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    
    print(f"\nModel Accuracy Evaluation:")
    print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    
    # Generate predictions on the entire dataset for more detailed metrics
    y_pred = model.predict(X)
    
    # Classification metrics
    print("\nClassification Report:")
    print(classification_report(y, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    return cv_scores.mean()


In [28]:
def test_model_with_new_venue(venue_description, formality_level, weather, time_of_day, 
                             venues, clothing_items, model, outfit_combinations, 
                             nn_model, outfit_features, outfit_ids, encoded_df_venues, encoded_df_items,
                             recommendation_rules, enc_venues):
    """Test the model with a new venue not in the original dataset"""
    # Create a new venue entry
    new_venue_id = f"V{len(venues) + 1:03d}"
    new_venue = pd.DataFrame({
        'venue_id': [new_venue_id],
        'venue_name': [venue_description],
        'formality_level': [formality_level],
        'description': [venue_description],
        'typical_weather': [weather],
        'time_of_day': [time_of_day]
    })
    
    # Encode the new venue features
    new_venue_features = pd.DataFrame({
        'typical_weather': [weather],
        'time_of_day': [time_of_day]
    })
    
    # One-hot encode the categorical features
    encoded_features = enc_venues.transform(new_venue_features)
    encoded_new_venue = pd.DataFrame(
        encoded_features, 
        columns=enc_venues.get_feature_names_out(['typical_weather', 'time_of_day'])
    )
    
    # Add formality level
    encoded_new_venue['formality_level'] = formality_level
    
    # Generate recommendations for the new venue
    print(f"\nRecommendations for new venue: {venue_description}")
    print(f"Formality Level: {formality_level}, Weather: {weather}, Time: {time_of_day}")
    
    # Generate item recommendations
    compatible_items = []
    
    for idx, item in clothing_items.iterrows():
        item_id = item['item_id']
        item_features = encoded_df_items.iloc[idx].values
        
        # Combine venue and item features
        combined_features = np.concatenate([encoded_new_venue.iloc[0].values, item_features])
        
        # Predict compatibility
        score = model.predict([combined_features])[0]
        compatible_items.append((item_id, score, item['item_name'], item['category']))
    
    # Sort by score and category
    compatible_items.sort(key=lambda x: (-x[1], x[3]))
    
    # Group by category and show top items
    categories = {}
    for item in compatible_items:
        if item[3] not in categories:
            categories[item[3]] = []
        if len(categories[item[3]]) < 2:  # Get top 2 for each category
            categories[item[3]].append(item)
    
    print("\nTop recommended items by category:")
    for category, items in categories.items():
        print(f"\n{category}:")
        for item in items:
            print(f"- {item[2]} (Compatibility Score: {item[1]})")
    
    # Create outfit recommendations by combining top items from different categories
    print("\nRecommended outfit combinations:")
    
    # Try to construct an outfit with top items from different categories
    outfit = []
    required_categories = ['Top', 'Bottom', 'Footwear']
    optional_categories = ['Outerwear', 'Suit', 'Dress']
    
    # Add required categories
    for category in required_categories:
        if category in categories and categories[category]:
            top_item = categories[category][0]
            outfit.append(top_item[2])
    
    # Add optional categories based on formality
    if formality_level >= 4:  # For formal events
        for category in optional_categories:
            if category in categories and categories[category]:
                if category == 'Dress' or category == 'Suit':  # These could replace Top+Bottom
                    if len(outfit) >= 2:  # If we already added Top and Bottom
                        outfit = [item for item in outfit if 'Top' not in item and 'Bottom' not in item]
                    outfit.append(categories[category][0][2])
                else:
                    outfit.append(categories[category][0][2])
    
    print(f"- Complete outfit: {', '.join(outfit)}")
    
    # Return the compatibility scores for analysis
    return compatible_items


In [31]:
def main():
    # Load data
    clothing_items, venues, compatibility_matrix, outfit_combinations, recommendation_rules, image_references = load_data()
    
    # Explore data
    pivot_comp = explore_data(clothing_items, venues, compatibility_matrix)
    
    # Preprocess data
    encoded_df_items, encoded_df_venues, enc_items, enc_venues = preprocess_data(clothing_items, venues)
    
    # Build models
    compatibility_model = build_compatibility_model(compatibility_matrix, clothing_items, venues, encoded_df_items, encoded_df_venues)
    nn_model, outfit_features, outfit_ids = build_outfit_recommender(clothing_items, encoded_df_items, outfit_combinations)
    
    # Evaluate model accuracy
    accuracy = evaluate_model_accuracy(compatibility_matrix, clothing_items, venues, encoded_df_items, encoded_df_venues, compatibility_model)
    
    # Analyze feature importance
    top_features = feature_importance(compatibility_model, encoded_df_items, encoded_df_venues)
    print("\nTop features for compatibility:")
    for feature, importance in top_features:
        print(f"- {feature}: {importance:.4f}")
    
    # Example recommendations
    print("\n" + "="*50)
    
    # Get user input for existing venue
    print("\nAvailable venues:")
    for _, venue in venues.iterrows():
        print(f"- {venue['venue_id']}: {venue['venue_name']} ({venue['description']})")
    
    while True:
        print("\nOptions:")
        print("1. Get recommendations for existing venue")
        print("2. Test with a new venue")
        print("3. Exit")
        
        choice = input("Enter your choice (1-3): ")
        
        if choice == '1':
            venue_id = input("\nEnter venue ID for outfit recommendations (e.g. V001): ")
            if venue_id in venues['venue_id'].values:
                recommend_outfit_for_venue(venue_id, venues, clothing_items, compatibility_model, outfit_combinations, 
                                          nn_model, outfit_features, outfit_ids, encoded_df_venues, encoded_df_items,
                                          recommendation_rules)
            else:
                print("Invalid venue ID. Please try again.")
        
        elif choice == '2':
            print("\nEnter details for a new venue:")
            venue_name = input("Venue name/description: ")
            
            # Get formality level with validation
            while True:
                try:
                    formality = int(input("Formality level (1-5, where 1 is casual and 5 is very formal): "))
                    if 1 <= formality <= 5:
                        break
                    else:
                        print("Please enter a value between 1 and 5.")
                except ValueError:
                    print("Please enter a valid number.")
            
            # Weather options
            print("\nWeather options:")
            print("1. Indoor")
            print("2. Outdoor Sunny")
            print("3. Outdoor Variable")
            print("4. Indoor Dark")
            
            weather_choice = input("Select weather (1-4): ")
            weather_map = {
                '1': 'Indoor', 
                '2': 'Outdoor Sunny', 
                '3': 'Outdoor Variable', 
                '4': 'Indoor Dark'
            }
            weather = weather_map.get(weather_choice, 'Indoor')
            
            # Time of day options
            print("\nTime of day options:")
            print("1. Daytime")
            print("2. Evening")
            print("3. Night")
            
            time_choice = input("Select time of day (1-3): ")
            time_map = {
                '1': 'Daytime', 
                '2': 'Evening', 
                '3': 'Night'
            }
            time_of_day = time_map.get(time_choice, 'Daytime')
            
            # Test with new venue
            test_model_with_new_venue(
                venue_name, formality, weather, time_of_day,
                venues, clothing_items, compatibility_model, outfit_combinations,
                nn_model, outfit_features, outfit_ids, encoded_df_venues, encoded_df_items,
                recommendation_rules, enc_venues
            )
        
        elif choice == '3':
            print("Exiting program.")
            break
        
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")